In [ ]:
import pandas as pd
import numpy as np
import os

# Paths
PROCESSED_PATH = "../data/processed/tabular/"
OUTPUT_FILE = os.path.join(PROCESSED_PATH, "supply_chain_features.csv")


# Load integrated dataset
df = pd.read_csv(os.path.join(PROCESSED_PATH, "integrated_supply_chain.csv"))
print("✅ Loaded integrated dataset:", df.shape)
df.head()